This example notebook shows how we can train an image classification model, as described [here](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb),
and store it as TileDB array. Firstly, let's import what we need.

In [3]:
import glob
import json
import os
import pickle
import shutil
from pprint import pprint

import tensorflow as tf
import tiledb
import matplotlib.pyplot as plt

from tiledb.ml.models.tensorflow_keras import TensorflowKerasTileDBModel


Load MNIST dataset for Keras datasets and scale.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

We can then define a function that creates a basic digit classifier for the MNIST dataset.

In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['accuracy'])

    return model

We can then train a model using some of our data. Let's assume that we initially train with the first 30000
observations from our dataset.

In [4]:
data_home = os.path.join(os.path.pardir, "data")
model = create_model()
cb_tb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(data_home, 'logs'))
cb_early = callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.01)

model.fit(x_train[:30000], y_train[:30000], epochs=5, callbacks=[cb_tb,cb_early])

2024-08-22 12:59:10.788587: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 12:59:10.833063: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2024-08-22 12:59:10.833094: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2024-08-22 12:59:10.833256: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2024-08-22 12:59:11.107514: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
 77/938 [=>............................] - ETA: 2s - loss: 1.0641 - accuracy: 0.6981

2024-08-22 12:59:11.703057: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2024-08-22 12:59:11.703101: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2024-08-22 12:59:11.710090: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2024-08-22 12:59:11.712768: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2024-08-22 12:59:11.729544: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ../data/logs/train/plugins/profile/2024_08_22_12_59_11

2024-08-22 12:59:11.731462: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ../data/logs/train/plugins/profile/2024_08_22_12_59_11/ktsitsi.trace.json.gz
2024-08-22 12:59:11.742256: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ../data/logs/train/plugins/profile/2024_08_22_12_59_11

2024-08-22

938/938 [==============================] - 3s 2ms/step - loss: 0.3811 - accuracy: 0.8888


We can now save the trained model as a TileDB array. In case we want to train  the model further in a later time, we can save
optimizer's information in our TileDB array. In case we will use our model only for inference, we don't have to save optimizer's
information and we only keep model's weights. We first declare a TileDB-Keras model object (with the corresponding uri and model attributes)
and then save the model as a TileDB array.

In [6]:
uri = os.path.join(data_home, 'keras-mnist-sequential-1')
tiledb_model_1 = TensorflowKerasTileDBModel(uri=uri, model=model)

tiledb_model_1.save(include_optimizer=True, callbacks=[cb_tb])

fragments_info = tiledb.array_fragments(uri)
print("number of fragments: {}".format(len(fragments_info)))

TypeError: save() got an unexpected keyword argument 'callbacks'

The above step will create a TileDB array in your working directory. For information about the structure of a dense
TileDB array in terms of files on disk please take a look [here](https://docs.tiledb.com/main/concepts/data-format).
Let's open our TileDB array model and check metadata. Metadata that are of type list, dict or tuple have been JSON
serialized while saving, i.e., we need json.loads to deserialize them.

In [ ]:
# Check array directory
pprint(glob.glob(f'{uri}/*'))

# Open in write mode in order to add metadata
model_array_1 = tiledb.open(uri)
for key, value in model_array_1.meta.items():
    print("Key: {}, Value: {}".format(key, value))

As we can see, in array's metadata we have by default information about the backend we used for training, keras version,
python version, model configuration and training configuration. We can load and check any of the aforementioned without
having to load the entire model in memory. Moreover, we can add any kind of extra information about model accuracy, model
version, deployment status etc, in the model's metadata either while saving the model, by passing a dictionary with any
kind of information, or by opening the TileDB array and adding new keys. Both cases are shown below.

In [ ]:
# Open the array in write mode
with tiledb.Array(uri, "w") as A:
    # Keep all history
    A.meta['loss'] = json.dumps(model.history.history['loss'])
    A.meta['accuracy'] = json.dumps(model.history.history['accuracy'])

    # Or keep last epoch's loss and accuracy
    A.meta['last_epoch_loss'] = json.dumps(model.history.history['loss'][-1])
    A.meta['last_epoch_accuracy'] = json.dumps(model.history.history['accuracy'][-1])

# Check that everything is there
model_array_1 = tiledb.open(uri)
for key, value in model_array_1.meta.items():
    if isinstance(value, bytes):
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value))

We can also save any kind of metadata while saving the model as a TileDB array, and avoid opening it multiple times.

In [ ]:
model = create_model()
model.fit(x_train[:30000], y_train[:30000], epochs=5)

uri2 = os.path.join(data_home, 'keras-mnist-sequential-2')
tiledb_model_2 = TensorflowKerasTileDBModel(uri=uri2, model=model)

tiledb_model_2.save(include_optimizer=True,
                    meta={"accuracy": model.history.history['accuracy'],
                          "loss": model.history.history['loss'],
                          "status": 'experimental'})

# Check array directory
print()
pprint(glob.glob(f'{uri2}/*'))

# Check that everything is there
print()
model_array_2 = tiledb.open(uri2)
for key, value in model_array_2.meta.items():
    if isinstance(value, bytes):
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value))

For the case of Tensorflow Keras models, apart from model configuration (architecture) which is saved in the metadata
section of the TileDB array, we save model's weights and optimizer's weights, as variable sized attributes (pickled),
i.e., we can open the TileDB and get only the weights of a model or model's optimizer without bringing the whole model in
memory. For example, we can load model's and optimizer's weights for models `keras-mnist-sequential-1` and
`keras-mnist-sequential-2` as follows.

In [ ]:
# Load model weights
model_1_weights = tiledb_model_1.get_weights()
model_2_weights = tiledb_model_2.get_weights()

# Load optimizer weights
optimizer_1_weights = tiledb_model_1.get_optimizer_weights()
optimizer_2_weights = tiledb_model_2.get_optimizer_weights()

plt.rcParams["figure.figsize"] = (8,10)

# Maybe plot a part of layer 1 weights for both NNs
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('1st 50x50 weights of layer 1 for NN1 and NN2', size=16, y=0.72)
ax1.matshow(model_1_weights[0][:50, :50])
ax2.matshow(model_2_weights[0][:50, :50])
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

# Maybe plot a part of optimizer weights for both NNs
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('1st 50x50 optimizer weights for NN1 and NN2', size=16, y=0.72)
ax1.matshow(optimizer_1_weights[2][:50, :50])
ax2.matshow(optimizer_2_weights[2][:50, :50])
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

Moving on, we can load the trained models for prediction or evaluation (we have to compile the model), as usual with
Tensorflow Keras models.

In [ ]:
loaded_model_1 = tiledb_model_1.load()
loaded_model_2 = tiledb_model_2.load()

# Make some predictions
pred_1 = loaded_model_1.predict(x_test)
pred_2 = loaded_model_2.predict(x_test)
print(type(pred_1), pred_1.shape)
print(type(pred_2), pred_2.shape)

# Evaluate models
loaded_model_1 = tiledb_model_1.load(compile_model=True)
loaded_model_2 = tiledb_model_2.load(compile_model=True)
loaded_model_1.evaluate(x_test, y_test)
loaded_model_2.evaluate(x_test, y_test)

What is really nice with saving models as TileDB array, is native versioning based on fragments as described
[here](https://docs.tiledb.com/main/concepts/data-format#immutable-fragments). We can load a model, retrain it
with new data and update the already existing TileDB model array with new model parameters and metadata. All information, old
and new will be there and accessible. This is extremely useful when you retrain with new data or trying different architectures
for the same problem, and you want to keep track of all your experiments without having to store different model instances. In our case,
let's continue training model_1 with the rest of our dataset and for 5 more epochs. After training is done, you will
notice the extra directories and files (fragments) added to `keras-mnist-sequential-1` TileDB array directory,
which keep all versions of the model.

In [ ]:
loaded_model_1 = tiledb_model_1.load(compile_model=True)
loaded_model_1.fit(x_train[30000:], y_train[30000:], epochs=5)

# and update
tiledb_model_1 = TensorflowKerasTileDBModel(uri=uri, model=loaded_model_1)
tiledb_model_1.save(include_optimizer=True,
                    meta={"accuracy": model.history.history['accuracy'],
                          "loss": model.history.history['loss'],
                          "version": '0.0.1',
                          "status": 'experimental'})

# Check array directory
print()
pprint(glob.glob(f'{uri}/*'))

# tiledb.array_fragments() requires TileDB-Py version > 0.8.5
fragments_info = tiledb.array_fragments(uri)

print()
print("====== FRAGMENTS  INFO ======")
print("array uri: {}".format(fragments_info.array_uri))
print("number of fragments: {}".format(len(fragments_info)))

for fragment_num, fragment in enumerate(fragments_info, start=1):
    print()
    print("===== FRAGMENT NUMBER {} =====".format(fragment.num))
    print("fragment uri: {}".format(fragment.uri))
    print("timestamp range: {}".format(fragment.timestamp_range))
    print(
        "number of unconsolidated metadata: {}".format(
            fragment.unconsolidated_metadata_num
        )
    )
    print("version: {}".format(fragment.version))

Finally, a very interesting and useful, for machine learning models, TileDB feature that is described
[here](https://docs.tiledb.com/main/concepts/data-format#groups) and [here](https://docs.tiledb.com/main/how-to/object-management#creating-tiledb-groups)
are groups. Assuming we want to solve the MNIST problem, and we want to try several architectures. We can save each architecture
as a separate TileDB array with native versioning each time it is re-trained, and then organise all models that solve the same problem (MNIST)
as a TileDB array group with any kind of hierarchy. Let's firstly define a new model architecture.

In [ ]:
def create_deeper_model():
    # For the sake of simplicity we just add an extra dense layer to the previous architecture.
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['accuracy'])

    return model

Then train it and save it as a new TileDB array.

In [ ]:
model = create_deeper_model()
model.fit(x_train, y_train, epochs=5)

uri3 = os.path.join(data_home, 'keras-mnist-sequential-deeper')
tiledb_deeper_model = TensorflowKerasTileDBModel(uri=uri3, model=model)

tiledb_deeper_model.save(include_optimizer=True,
                         meta={"accuracy": model.history.history['accuracy'],
                               "loss": model.history.history['loss'],
                               "status": 'experimental'})

# Check array directory
print()
pprint(glob.glob(f'{uri3}/*'))

Now we can create a TileDB group and organise (in hierarchies, e.g., sophisticated vs less sophisticated) all our
MNIST models as follows.

In [ ]:
group = os.path.join(data_home, 'tiledb-keras-mnist')
tiledb.group_create(group)
shutil.move(uri, group)
shutil.move(uri2, group)
shutil.move(uri3, group)

Any time we can check and query all the available models, including their metadata, for a specific problem like MNIST.

In [ ]:
tiledb.ls(group, lambda obj_path, obj_type: print(obj_path, obj_type))